In [1]:
# This is the combined code i.e all the learnings are applied
# togeather and used to form the whole neural n/w

In [2]:
# Imports
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

In [3]:
# Dense Layer
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        #Initialize weights and biases
        #np.random.randn is a gaussian distribution which will provide values between -1 and 1 and 0.01 is multiplied to make the values smaller 
        # and close to each other thus fitting of data will be easier during training
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons) #To avoid transposing every time (inputs, neurons) is used rather than (neurons, inputs) 
        self.biases = np.zeros((1, n_neurons)) #default bias initialization
        pass
        
    def forward(self, inputs):
        #Calculate output values from inputs,weights and biases
        print("Weights are = ",self.weights)
        print("Biases are = ",self.biases)
        self.output = np.dot(inputs, self.weights) + self.biases
        pass

In [4]:
# Relu Activation
class Activation_ReLU:
    def activate(self, inputs):
        self.output = np.maximum(0,inputs)

In [5]:
# Softmax Activation
class Activation_Softmax:
    #Forward pass
    def activate(self, inputs):
        #Get unnormalized input batch (inputs - max(input)). It is done so that the values if they are bigger like in the order of 1000 then e^1000
        # calculation may fail as it can overflow. So keeping the inputs in a small -ve range will give a +ve output with values between 0 and 1
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        #Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

In [6]:
# Common Loss Class uses Cross Entropy Loss
class Loss:
    def calculate(self, output, y):

        # calculate sample losses
        sample_losses = self.forward(output, y)

        # calculate mean loss
        data_loss = np.mean(sample_losses)

        #return loss
        return data_loss

In [14]:
# Cross Entropy Loss
class Loss_CategoricalCrossEntropy(Loss):

    #Forward Pass

    def forward(self, y_pred, y_true):
        # No of samples in a batch
        samples = len(y_pred)

        # clip the data to avoid log(0) error
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        avg_loss = 0.0
        # checking whether the class target/real values are sparse labels or one hot encoded
        if(len(y_true.shape) == 1):
            output = []
            for targ_idx, distribution in zip(y_true, y_pred_clipped):
                output.append(distribution[targ_idx])

            loss_list = -np.log(output)
            #Calculating the Average loss
            avg_loss = np.mean(loss_list)
            #print("Avg Loss = ", avg_loss)
            
        elif(len(y_true.shape) == 2):
            outputs = []
            sum = 0
            for i in range(0,samples):
                sum = 0
                for k in range(0,len(y_true[0])):
                    sum = sum + y_true[i][k]*np.log(y_pred_clipped[i][k])
                outputs.append(sum)

            avg_loss = -1/N * np.sum(outputs)
            #print("Avg Loss = ",avg_loss)
        
        return avg_loss

In [15]:
def main():
    #Create dataset
    X, y = spiral_data(samples=100, classes=3)
    
    dense_layer1 = Layer_Dense(2, 3)

    dense_layer1.forward(X)
    
    activation1 = Activation_ReLU() #Activation function 

    activation1.activate(dense_layer1.output) #Activates the neurons using ReLU and it's not normalized

    dense_layer2 = Layer_Dense(3,3)

    dense_layer2.forward(activation1.output)
    
    activation2 = Activation_Softmax()
    activation2.activate(dense_layer2.output) #Passing through softmax activation at the output layer
    
    print("Dense Layer 2 Output After Softmax Activation::")
    print(activation2.output[:5]) # output from the activation function

    loss_function = Loss_CategoricalCrossEntropy()
    loss = loss_function.calculate(activation2.output, y)
    print('loss:', loss)

In [16]:
main()

Weights are =  [[-0.00189899  0.01188112  0.00603885]
 [ 0.01809998 -0.00730415 -0.0048045 ]]
Biases are =  [[0. 0. 0.]]
Weights are =  [[ 0.02523316 -0.00779414 -0.00210855]
 [ 0.01729084  0.00037448  0.01480814]
 [ 0.00067967 -0.00054518 -0.00270281]]
Biases are =  [[0. 0. 0.]]
Dense Layer 2 Output After Softmax Activation::
[[0.33333333 0.33333333 0.33333333]
 [0.33333454 0.33333256 0.3333329 ]
 [0.3333357  0.33333181 0.33333248]
 [0.33333616 0.33333146 0.33333239]
 [0.33333826 0.33333017 0.33333157]]
loss: 1.0986117131151507


In [1]:
#Accuracy Calculation
# Accuracy describes how often the largest confidence is the correct class in terms of
# fraction.
# To find the accuracy we will take the argmax value from the softmax outputs and then compare
# these to the targets.

In [7]:
import numpy as np

# Probabilities of 3 samples
softmax_outputs = np.array([[0.7, 0.2, 0.1],
                            [0.5, 0.1, 0.4],
                            [0.02, 0.9, 0.08]])

# Target (ground-truth) labels for 3 samples
class_targets = np.array([0,1,1])

# Calculate the values along the second axis(rows) and returns the index of that element
predictions = np.argmax(softmax_outputs, axis=1)

# If the targets are one-hot encoded -convert them
if len(class_targets.shape) == 2:
    class_targets = np.argmax(class_targets, axis=1)
# True evaluates to 1 False = 0
accuracy = np.mean(predictions==class_targets)

print("Predictions = ",predictions)
print("Truth = ",class_targets)
print("Accuracy: ", accuracy)


Predictions =  [0 0 1]
Truth =  [0 1 1]
Accuracy:  0.6666666666666666
